Sampling single mode initial states using methods presented in the article
- [Olsen, Bradley, Opt. Comm. 282 (2009) 3924-3929](https://doi.org/10.1016/j.optcom.2009.06.033); errata [Olsen, Lewis-Swan, Bradley, Opt. Comm. 370 (2016) 327-328](https://doi.org/10.1016/j.optcom.2016.02.068)

We also sample some new distributions:
- fock state for +W
- thermal state for +P

More detailed errata for [Olsen, Bradley, Opt. Comm. 282 (2009) 3924-3929](https://doi.org/10.1016/j.optcom.2009.06.033)


In [ ]:
using PhaseSpaceTools, PyPlot, QuadGK

In [ ]:
b=10
N=1000
a,ā = coherent(b,N,dist="W")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā)-.5)

In [ ]:
a,ā = thermal(0,10,5000;dist="P")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
mean(a.*ā)

In [ ]:
a,ā = thermal(0,10,5000;dist="W")
figure(figsize=(2,2))
scatter(real(a),imag(ā),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
mean(a.*ā)-.5

In [ ]:
β = 10
ϕ = π/16
r = 1.5
ϵ = r*exp(2*im*ϕ)
N = 5000
a,ā = squeezed(β,ϵ,N,dist="W")
figure(figsize=(2,2))
scatter(real(a),imag(ā),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā)-.5)
println(sinh(abs(ϵ)).^2+abs2(β))

In [ ]:
β = 10
ϕ = π/16
r = 2
ϵ = r*exp(2*im*ϕ)
N = 100000
a,ā = squeezed(β,ϵ,N;dist="+P")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(real(mean(a.*ā)))
println(sinh(abs(ϵ)).^2+abs2(β))

In [ ]:
n = 100
N = 100000
a,ā = fock(n,N;dist="W")
figure(figsize=(2,2))
scatter(real(a),imag(ā),s=.5,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā)-.5)
println(n)

In [ ]:
n = 160
N = 10000
a,ā = fock(n,N;dist="+P")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=.5,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā))
println(n)

In [ ]:
β = 10
ϵ = 0
q = .5
N = 5000
a,ā = crescent(β,ϵ,q,N;dist="W")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā)-.5)
#not quite the right moments for crescent state, but a sanity check of sorts:
println(sinh(abs(ϵ)).^2+abs2(β)) 


In [ ]:
β = 10
ϵ = 0
q = .4
N = 5000
a,ā = crescent(β,ϵ,q,N;dist="Q")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā)-1)
println(sinh(abs(ϵ)).^2+abs2(β)) #not quite the right moments for crescent state, but a check of sorts

In [ ]:
β = 10
ϵ = 0
q = .4
N = 5000
a,ā = crescent(β,ϵ,q,N;dist="+P")
figure(figsize=(2,2))
scatter(real(a),imag(a),s=1,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā))
println(sinh(abs(ϵ)).^2+abs2(β)) #not quite the right moments for crescent state, but a check of sorts

# implement fock states for +W

In [ ]:
function reject(P,w,N,Pmax)
    #rejection sampling the probability distribution P over the window w=[w1,w2]
    #Pmax must be numerical with value max(P) <= Pmax 
    #initialize sample vector
    samples = Array{Float64}(1)
    while length(samples) < N + 1 
        y = w[1] + rand()*(w[2]-w[1])
        z = rand()*Pmax
        if z < P(y)
            push!(samples,y)
        end
    end
    return samples[2:end]
end
    


In [ ]:
f(x)=0.5*(exp(-x^2/2)/sqrt(2π)+exp(-(x-10)^2/8)/sqrt(8π))

In [ ]:
weighted_hist(x; kws...) = PyPlot.plt[:hist](x; weights=ones(length(x))/length(x), kws...)

In [ ]:
a = reject(f,[-5,18],100000,f(0.))
figure(figsize=(6,2))
b = weighted_hist(a,bins=60);

In [ ]:
mean(a),sqrt(var(a))

### Distribution to sample

In [ ]:
@pyimport mpmath as mp
lagbig(x,n)=mp.laguerre(n,0,x)

In [ ]:
@pyimport scipy as sp
@pyimport scipy.special as sps
laguerre(x,n)=sps.eval_laguerre(n,x)

In [ ]:
lagbig(.1,10000)

In [ ]:
laguerre(.1,10000)

Distribution 
$$P_n(x)=\frac{4}{3}e^{-2x^2/3}L_n(-4x^2/3)\frac{x}{3^n}$$
or, using 
$$L_n(-x)=\sum_{k=0}^n\binom{n}{k}\frac{x^k}{k!}$$
we have
$$P_n(x)=\frac{4x}{3^{n+1}}e^{-2x^2/3}\sum_{k=0}^n\binom{n}{k}\frac{(4x^2/3)^k}{k!}$$

In [ ]:
function P(x,n)
    #P(x,n)=(4/3)*exp(-2*x^2/3)*x*laguerre(-4*x^2/3,n)/(3^n)
    return (4/3)*exp(-2*x^2/3 + log(x) + log(laguerre(-4*x^2/3,n)) - n*log(3))
end
n=10
println(sqrt(n))
println(quadgk(x->P(x,n),0,10))
x̄=quadgk(x->P(x,n).*x,0,10);println(x̄)
x2=quadgk(x->P(x,n).*x.^2,0,10)
sigx=sqrt(x2[1]-x̄[1].^2)
x̄[1]+6sigx,x̄[1]-6sigx

Asymptotic expansion
$$L_n(-x)=\frac{e^{-x/2}}{2\sqrt{\pi}[x(n+1)]^{1/4}}e^{2\sqrt{x(n+1)}}$$

and hence
$$P_n(x)=\frac{4}{3}e^{-2x^2/3}L_n(-4x^2/3)\frac{x}{3^{n}}\longrightarrow \frac{4}{3}e^{-2x^2/3}\frac{e^{-2x^2/3}}{2\sqrt{\pi}[4x^2(n+1)/3]^{1/4}}e^{2\sqrt{4x^2/3(n+1)}}\frac{x}{3^{n}}$$
or 
$$P_n(x)\longrightarrow \frac{2}{3\sqrt{\pi}}\frac{e^{-(\sqrt{4x^2/3}-\sqrt{n+1})^2}}{[4x^2(n+1)/3]^{1/4}}\frac{xe^{n+1}}{3^{n}}$$

In [ ]:
n=0
x=linspace(0,25,1000);dx=x[2]-x[1];
plot(x,P.(x,n));
x1=max(0,sqrt(n+1)-5);x2=sqrt(n+1)+5
plot(sqrt(n+1)*ones(x),linspace(0,.6,1000))
plot(x1*ones(x),linspace(0,.6,1000))
plot(x2*ones(x),linspace(0,.6,1000))
sum(P.(x,n)*dx)
maximum(P.(x,n))

In [ ]:
n=1
x1=max(0,sqrt(n)-5);x2=sqrt(n)+5
N=10000
Nb=50
a = reject(x->P.(x,n),[x1,x2],N,.6)
figure(figsize=(7,3))
b = weighted_hist(a,bins=Nb);

In [ ]:
import PhaseSpaceTools.fock

weighted_hist(x; kws...) = PyPlot.plt[:hist](x; weights=ones(length(x))/length(x), kws...)

function reject(P,w,N,Pmax)
    #rejection sampling the probability distribution P over the window w=[w1,w2]
    #Pmax must be numerical with value max(P) <= Pmax 
    #initialize sample vector
    samples = Array{Float64}(1)
    while length(samples) < N + 1 
        y = w[1] + rand()*(w[2]-w[1])
        z = rand()*Pmax
        if z < P(y)
            push!(samples,y)
        end
    end
    return samples[2:end]
end

@pyimport scipy as sp
@pyimport scipy.special as sps
laguerre(x,n)=sps.eval_laguerre(n,x)

function P(x,n)
#P(x,n)=(4/3)*exp(-2*x^2/3)*x*laguerre(-4*x^2/3,n)/(3^n)
    return (4/3)*exp(-2*x^2/3 + log(x) + log(laguerre(-4*x^2/3,n)) - n*log(3))
end

In [ ]:
"""
```julia
a,ā = fock(n,N;dist)
```
samples phase space distribution for a Fock state
`n` is number of fock state

`N` is number of samples

`dist` is distribution. Can be either `W` or `+P`(default)
"""
function fock(n,N;dist="+P")
if dist=="+P"
    γ = (randn(N)+im*randn(N))/sqrt(2)
    d = Gamma(n+1,1)
    z = rand(d,N)
    μ = sqrt.(z).*exp.(2π*im*rand(N))
    α = μ + γ
    ᾱ = conj(μ - γ)
    return α, ᾱ
elseif dist=="W"
    p = 0.5*sqrt(2*n+1+2*sqrt(n^2+n))
    q = 1/(4*p)
    α = (p + q*randn(N)).*exp.(2π*im*rand(N))
    ᾱ = conj(α)
    return α,ᾱ
elseif dist=="+W" 
    γ = (randn(N)+im*randn(N))/sqrt(2)
    x1= max(0,sqrt(n)-5); x2 = sqrt(n)+5
    (n==0||n==1)?Pmax=0.71:Pmax=0.6
    z = reject(x->P.(x,n),[x1,x2],N,Pmax)
    μ = z.*exp.(2π*im*rand(N))
    α = μ + γ
    ᾱ = conj(μ - γ)
    return α, ᾱ    
else error("distribution not implemented")
        
end
end

In [ ]:
a,ā = fock(0,100000,dist="+W")

In [ ]:
n̄ = mean(a.*ā)-.5

In [ ]:
Vn = mean(a.^2.*ā.^2) - n̄ - n̄^2 #+P

In [ ]:
n = 20
N = 5000
a,ā = fock(n,N)
figure(figsize=(2,2))
scatter(real(a),imag(a),s=.5,c="blue")
axis("square")
axis([-20,20,-20,20]);
println(mean(a.*ā))
println(n)